In [1]:
# 파이썬 패키지 호출 
import hashlib
import json
from time import time # timestamp를 사용자 친화적으로
import random
import requests # https 요청시 사용되는 모듈 (Flask로 구축된 블록체인 API와 소통)
from flask import Flask, request, jsonify

In [2]:
# 블록체인 객체 만들기 
class Blockchain(object):
    def __init__(self): # 블록체인 객체 생성
        self.chain = [] # 블록들이 저장되는 체인
        self.current_transaction = [] # 블록 내에 저장될 거래 내역
        self.nodes = set() # 노드 정보
        self.new_block(previous_hash=1, proof=100)
    
    # 해시암호화 함수 
    # json 형식의 거래내역들이 SHA-256 방식으로 해시
    @staticmethod
    def hash(block):
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()
    
    # 마지막 블록 호출 함수 
    # 최근 논스값을 기반으로 새로운 논스 값을 찾는다 - 가장 마지막 블록을 호출하는 함수
    @property
    def last_block(self):
        return self.chain[-1]
    
    # 블록 검증함수: 마지막 블록의 논스 값과 신규 논스 값 후보를 결합하여 해시화
    # 첫 4개 단어가 0000 일때 해당 논스값이 유효하다고 판단
    @staticmethod
    def valid_proof(last_block, proof):
        guess = str(last_block + proof).encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == "0000"
    
    # PoW 함수 
    # 난수값을 생성해 가장 최근 블록의 논스값과 비교하여 PoW가 성공할 때까지 반복
    def pow(self, last_proof):
        proof = random.randint(-1000000, 1000000)
        while self.valid_proof(last_proof, proof) is False:
            proof = random.randint(-1000000,1000000)
        return proof 
    
    # 거래 내역 추가 함수 
    def new_transaction(self, sender, recipient, amount):
        self.current_transaction.append(
            {
                'sender' : sender, # 송신자
                'recipient' : recipient, # 수신자
                'amount' : amount, # 금액
                'timestamp':time()
            }
        )
        return self.last_block['index'] + 1
    
    # 블록 추가 함수 (신규 블록 생성)
    # 신규 블록이 생성될 때 필요한 인자 5가지 - 블록 번호, 생성시간, 거래내역, 논스값, 이전 블록의 해시
    # 기존의 거래 내역이 저장된 후 거래 내역 리스트는 초기화, 생성된 블록은 객체의 체인 리스트에 추가
    def new_block(self, proof, previous_hash=None):
        block = {
            'index' : len(self.chain)+1,
            'timestamp' : time(), # timestamp from 1970
            'transactions' : self.current_transaction,
            'nonce' : proof,
            'previous_hash' : previous_hash or self.hash(self.chain[-1]),
        }
        self.current_transaction = []
        self.chain.append(block)     
        return block
    
    # 블록 검증
    def valid_chain(self, chain):
        last_block = chain[0]
        current_index = 1

        while current_index < len(chain):
            block = chain[current_index]
            print('%s' % last_block)
            print('%s' % block)
            print("\n----------\n")
            if block['previous_hash'] != self.hash(last_block):
                return False 
            last_block = block
            current_index += 1
        return True


In [3]:
# 블록체인 객체 기반으로 노드 만들기 

# 노드 기본 정보 설정
blockchain = Blockchain() # 블록체인 객체를 호출 
my_ip = '0.0.0.0'
my_port = '5001' # 포트 사용중이면 다른 포트로 변경 가능 
node_identifier = 'node_' + my_port
mine_owner = 'master' # 채굴 결과 발생하는 수익을 보낼 지갑 주소
mine_profit = 0.1 # 채굴 보상값

app = Flask(__name__)

# 블록 정보 호출 (full_chain)
# 투명성: 임의의 사용자가 블록체인 정보 호출 시 블록체인 내의 블록 길이와 블록의 모든 정보를 json 양식으로 리턴
@app.route('/chain', methods=['GET'])
def full_chain():
    print("chain info requested!!")
    response = {
        'chain' : blockchain.chain,
        'length': len(blockchain.chain)
    }
    return jsonify(response), 200

# 신규 거래 추가 (new_transaction)
@app.route('/transactions.new', methods=['POST'])
def new_transaction():
    values = request.get_json() # 사용자의 거래가 발생할 경우 해당 거래 내역은 json 형식으로 요청됨
    print("transactions_new!!! : ", values)
    required = ['sender', 'recipient', 'amount']

    # 요청 사항에 거래 내역의 3가지 요소 (발신자, 수신자, 보내는 금액)이 있는지 확인 (없으면 404에러)
    if not all(k in values for k in required):
        return 'missing values', 400
    
    index = blockchain.new_transaction(values['sender'], values['recipient'], values['amount'])
    response = {'message' : 'Transaction will be added to Block {%s}' % index}

    return jsonify(response),201

# 채굴 
@app.route('/mine', methods=['GET'])
def mine():
    # 채굴이 시작되면 마지막 블록의 논스값을 블록 객체의 pow에 넣고 작업 시작
    print("MINING STARTED")
    last_block = blockchain.last_block
    last_proof = last_block['nonce']
    proof = blockchain.pow(last_proof)

    # 채굴 보상
    blockchain.new_transaction(
        sender = mine_owner,
        recipient = node_identifier,
        amount = mine_profit # coinbase transaction
    )

    # 전 블록의 해시값을 포함하여 블록 객체의 new_block 함수로 블록이 생성됨
    previous_hash = blockchain.hash(last_block)
    block = blockchain.new_block(proof, previous_hash)
    print("MININT FINISHED")

    response = {
        'message' : 'new block found',
        'index' : block['index'],
        'transactions' : block['transactions'],
        'nonce' : block['nonce'],
        'previous_hash' : block['previous_hash']
    }

    # 정상적으로 처리되면 json 결과값으로 리턴
    return jsonify(response), 200

# 노드 운영
app = Flask(__name__)
if __name__ == '__main__':
    app.run(host=my_ip, port=my_port)

# 모든 코드를 실행하면 블록체인 내의 __init__ 함수가 실행되어 genesis block이 생성된다
# 성공적으로 돌려지면 running on ~ 이렇게 나온다


NameError: name 'app' is not defined